# A ReAct Agent with Command R+, Integrating a Custom API Tool for Financial Analysis

Adaptive RAG is a strategy for RAG that unites (1) [query analysis](https://blog.langchain.dev/query-construction/) with (2) [active / self-corrective RAG](https://blog.langchain.dev/agentic-rag-with-langgraph/).

In the paper, they report query analysis to route across:
- No Retrieval (LLM answers)
- Single-shot RAG
- Iterative RAG


We'll use Command R+, a recent release from Cohere that:
- Has strong accuracy on RAG, Tool Use and Agents
- Has 128k context


# Environment

In [1]:
%pip install --quiet langchain langchain_cohere tiktoken faiss-cpu beautifulsoup4 langchain_experimental matplotlib tabulate python-dotenv

Note: you may need to restart the kernel to use updated packages.


Save your credentials in a .env file in your user root directory, so that you can retrieve securely

In [2]:
import os
from dotenv import load_dotenv
load_dotenv()

COHERE_API_KEY = os.environ.get("COHERE_API_KEY")
TAVILY_API_KEY = os.environ.get("TAVILY_API_KEY") # Get your Free API key at https://app.tavily.com once you sign up
FMP_API_KEY = os.environ.get("FMP_API_KEY") # Get your Free API key https://site.financialmodelingprep.com/ once you sign up

# Create tools
The ReAct agent will be equipped with these tools. The model can pick between
- web search
- RAG: retrieval from a vector store
- custom tool (call an external API)
- directly answering

The model can use any of these tools, in any sequence of steps, and self-reflect.

### Web search tool

In [3]:
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain_core.pydantic_v1 import BaseModel, Field

internet_search = TavilySearchResults()
internet_search.name = "internet_search"
internet_search.description = "Returns a list of relevant document snippets for a textual query retrieved from the internet."


class TavilySearchInput(BaseModel):
    query: str = Field(description="Query to search the internet with")


internet_search.args_schema = TavilySearchInput

### Python interpreter tool

In [4]:
from langchain.agents import Tool
from langchain_experimental.utilities import PythonREPL

python_repl = PythonREPL()
repl_tool = Tool(
    name="python_repl",
    description="Executes python code and returns the result. The code runs in a static sandbox without interactive mode, so print output or save output to a file.",
    func=python_repl.run,
)
repl_tool.name = "python_interpreter"

# from langchain_core.pydantic_v1 import BaseModel, Field
class ToolInput(BaseModel):
    code: str = Field(description="Python code to execute.")
repl_tool.args_schema = ToolInput

### RAG tool

In [5]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_cohere import CohereEmbeddings
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import FAISS

# Set embeddings
embd = CohereEmbeddings()

# Docs to index
urls = [
    "https://www.mayerbrown.com/en/insights/publications/2023/03/new-data-standards-pending-for-federally-regulated-financial-entities",
    "https://plaid.com/resources/open-finance/what-is-fdx/",
    "https://www.egnyte.com/guides/financial-services/financial-data-compliance",
]

# Load
docs = [WebBaseLoader(url).load() for url in urls]
docs_list = [item for sublist in docs for item in sublist]

# Split
text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
    chunk_size=512, chunk_overlap=0
)
doc_splits = text_splitter.split_documents(docs_list)

# Add to vectorstore
vectorstore = FAISS.from_documents(
    documents=doc_splits,
    embedding=embd,
)

vectorstore_retriever = vectorstore.as_retriever()

In [6]:
from langchain.tools.retriever import create_retriever_tool

vectorstore_search = create_retriever_tool(
    retriever=vectorstore_retriever,
    name="vectorstore_search",
    description="Retrieve relevant info from a vectorstore that contains documents related to Data Compliance for Financial Services and its regulation.",
)

### Custom Tool For Market Capitalization



1. Define the function

In [7]:
import requests

# Function to get the market capitalization of a ticker symbol
def get_market_cap(ticker):
    url = f"https://financialmodelingprep.com/api/v3/market-capitalization/{ticker}?apikey={FMP_API_KEY}"
    response = requests.get(url)
    if response.status_code == 200:
        data = response.json()
        if data and isinstance(data, list) and len(data) > 0:
            market_cap = data[0].get('marketCap', 'No market cap data available')
            return [{'id': 0, 'text': f'Market cap for {ticker}: ${market_cap}'}]
        else:
            return "No data available for the specified ticker."
    else:
        return "Failed to retrieve data from the API."

2. Define the Custom Tool

In [8]:
from langchain.tools import tool

@tool
def market_cap(ticker: str) -> list:
    """This tool is only used to find market capitalization. Do not use it for anything else. Find the ticker when asked about capitalization. The ticker symbol of the company (e.g., AAPL which is the ticker for Apple Inc, MSFT for Microsoft )"""
    return get_market_cap(ticker)
    

# Create the ReAct Agent
The model can smartly pick the right tool(s) for the user query, call them in any sequence of steps, analyze the results and self-reflect.

In [9]:
from langchain.agents import AgentExecutor
from langchain_cohere.react_multi_hop.agent import create_cohere_react_agent
from langchain_core.prompts import ChatPromptTemplate

In [10]:
# LLM
from langchain_cohere.chat_models import ChatCohere

chat = ChatCohere(model="command-r-plus", temperature=0.3)

# Preamble
preamble = """
Use all tools that are available to answear the question. 
If the query covers the topics of Federal Financial Institutions, use the vectorstore search first.
If the query covers market capitalization, use the market cap tool first.
You are equipped with an internet search tool, and python interpreter, a market cap api, and a special vectorstore of information about Data Compliance for Financial Services.

"""

# Prompt
prompt = ChatPromptTemplate.from_template("{input}")

# Create the ReAct agent
agent = create_cohere_react_agent(
    llm=chat,
    tools=[vectorstore_search, internet_search, repl_tool, market_cap],
    prompt=prompt,
)

In [11]:
agent_executor = AgentExecutor(
    agent=agent, tools=[vectorstore_search, internet_search, repl_tool, market_cap], verbose=True
)

# Testing the ReAct agent

**Let's ask a question that requires web search.**

In [12]:
result = agent_executor.invoke(
    {
        "input": "How does the current interest rate environment affect the bond market?",
        "preamble": preamble,
    }
)



> Entering new AgentExecutor chain...

I will search for how the current interest rate environment affects the bond market.
{'tool_name': 'internet_search', 'parameters': {'query': 'How does the current interest rate environment affect the bond market?'}}
[{'url': 'https://www.morningstar.com/bonds/how-do-interest-rates-affect-your-bonds', 'content': "The federal-funds rate, the interest rate at which banks lend money to each other overnight, is now targeted between 1.75% and 2.00%. When the Fed raises or lowers rates, it affects bonds' prices ..."}, {'url': 'https://www.usbank.com/investing/financial-perspectives/market-news/interest-rates-affect-bonds.html', 'content': 'Personal\nBank accounts\nCredit cards\nInvesting and retirement\nPersonal loans & lines\nHome loans\nVehicle loans\nMobile and online\nExplore personal banking\nWealth Management\nOur services\nInvesting\nYour goals\nAdvisors & wealth teams\nPrivate Wealth Management\nOur perspectives\nExplore Wealth Management\nBus

In [13]:
result["output"]

'Interest rates have a direct impact on bond prices. When interest rates rise, bond prices fall, and vice versa. The federal-funds rate, the interest rate at which banks lend money to each other overnight, is now targeted between 1.75% and 2.00%. The Fed sought to slow the economy as a way to reduce inflation, which peaked at 9.1% for the 12 months ending June 2022, but dropped to less than 3.1% by November 2023.'

**Let's ask a question that requires RAG retrieval from the vector db.**

In [14]:
result = agent_executor.invoke(
    {
        "input": "What are the primary functions of the Federal Financial Institutions Examination Council (FFIEC)?",
        "preamble": preamble,
    }
)



> Entering new AgentExecutor chain...

I will search for the primary functions of the Federal Financial Institutions Examination Council (FFIEC) in the vector store.
{'tool_name': 'vectorstore_search', 'parameters': {'query': 'primary functions of the Federal Financial Institutions Examination Council (FFIEC)?'}}
losses and improve overall cybersecurity by implementing best practices and employing proven cybersecurity frameworks. The importance of financial data compliance continues to grow as regulations continue to expand globally to ensure that customers’ personal information and finances are protected from theft and fraud.Advantages of Financial Data Compliance and Cost of Non-ComplianceBenefits of financial data complianceNot following financial data complianceView of the most critical data and systemsPlan for cybersecurity tools and practicesProtection of valuable informationRapid response to cybersecurity incidentsOperational disruptionsReputational damageCivil and criminal la

In [15]:
result["output"]

'The Federal Financial Institutions Examination Council (FFIEC) is a five-member U.S. government interagency body, consisting of five banking regulators. It is responsible for uniformity in the guidelines and supervision of financial institutions, including financial data compliance. It includes financial data management mandates, such as directing how organisations should govern the secure storage of all types of sensitive information, whether on computer systems, physical media, or hard-copy documents.'

**Let's ask a question that requires the custom Market Cap tool.**

In [16]:
result = agent_executor.invoke(
    {
        "input": "What is Apple capitalization compared to Microsoft? Calculate the difference",
         "preamble": preamble,
    }
)



> Entering new AgentExecutor chain...

I will use the market_cap tool to find the market capitalization of Apple and Microsoft. Then, I will use Python to calculate the difference between the two.
{'tool_name': 'market_cap', 'parameters': {'ticker': 'AAPL'}}
[{'id': 0, 'text': 'Market cap for AAPL: $2927279690000'}]
{'tool_name': 'market_cap', 'parameters': {'ticker': 'MSFT'}}
[{'id': 0, 'text': 'Market cap for MSFT: $3023872485050'}]

Python REPL can execute arbitrary code. Use with caution.



The market capitalization of Apple is $2,927,279,690,000 and the market capitalization of Microsoft is $3,023,872,485,050. I will now use Python to calculate the difference between the two.
{'tool_name': 'python_interpreter', 'parameters': {'code': 'apple_market_cap = 2927279690000\nmicrosoft_market_cap = 3023872485050\n\ndifference = abs(microsoft_market_cap - apple_market_cap)\n\nprint(f"The difference in market capitalization between Apple and Microsoft is {difference} dollars.")'}}
The difference in market capitalization between Apple and Microsoft is 96592795050 dollars.
Relevant Documents: 0,1,2
Cited Documents: 0,1,2
Answer: The market capitalization of Apple is $2,927,279,690,000 and the market capitalization of Microsoft is $3,023,872,485,050. The difference in market capitalization between the two companies is $96,592,795,050.
Grounded answer: The market capitalization of Apple is <co: 0>$2,927,279,690,000</co: 0> and the market capitalization of Microsoft is <co: 1>$3,023,8

In [17]:
result["output"]

'The market capitalization of Apple is $2,927,279,690,000 and the market capitalization of Microsoft is $3,023,872,485,050. The difference in market capitalization between the two companies is $96,592,795,050.'

**Let's ask a question that does not require any tools.**

In [18]:
result = agent_executor.invoke(
    {
        "input": "Hi there!",
         "preamble": preamble,
    }
)



> Entering new AgentExecutor chain...

I will respond to the user's greeting.
{'tool_name': 'directly_answer', 'parameters': {}}
directly_answer is not a valid tool, try one of [vectorstore_search, internet_search, python_interpreter, market_cap].Relevant Documents: None
Cited Documents: None
Answer: Hi! How can I help?
Grounded answer: Hi! How can I help?

> Finished chain.


In [19]:
result["output"]

'Hi! How can I help?'